In [1]:
!pip install numpy pandas matplotlib scikit-learn tensorflow

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

2024-06-13 14:32:53.627964: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 14:32:54.721045: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-13 14:32:57.077238: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-13 14:33:00.244608: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [ ]:
class LSTMCellWithDropout(layers.LSTMCell):
    def __init__(self, units, dropout_rate=0.5, **kwargs):
        super(LSTMCellWithDropout, self).__init__(units, **kwargs)
        self.dropout_rate = dropout_rate

    def call(self, inputs, states, training=None):
        h_tm1, c_tm1 = states  # previous hidden state and cell state

        if training:
            # Generate dropout mask
            dropout_mask = tf.keras.backend.random_binomial(
                shape=tf.shape(h_tm1),
                p=1 - self.dropout_rate
            )
            # Apply dropout mask
            h_tm1 = dropout_mask * h_tm1

        # Forget gate
        f = tf.keras.backend.sigmoid(
            tf.keras.backend.dot(inputs, self.recurrent_kernel[:, :self.units]) +
            tf.keras.backend.dot(h_tm1, self.kernel[:, :self.units]) + self.bias[:self.units]
        )

        # Input gate
        i = tf.keras.backend.sigmoid(
            tf.keras.backend.dot(inputs, self.recurrent_kernel[:, self.units:self.units * 2]) +
            tf.keras.backend.dot(h_tm1, self.kernel[:, self.units:self.units * 2]) + self.bias[self.units:self.units * 2]
        )

        # Cell state
        C_tilde = tf.keras.backend.tanh(
            tf.keras.backend.dot(inputs, self.recurrent_kernel[:, self.units * 2:self.units * 3]) +
            tf.keras.backend.dot(h_tm1, self.kernel[:, self.units * 2:self.units * 3]) + self.bias[self.units * 2:self.units * 3]
        )

        # New cell state
        C_t = f * c_tm1 + i * C_tilde

        # Output gate
        o = tf.keras.backend.sigmoid(
            tf.keras.backend.dot(inputs, self.recurrent_kernel[:, self.units * 3:]) +
            tf.keras.backend.dot(h_tm1, self.kernel[:, self.units * 3:]) + self.bias[self.units * 3:]
        )

        # New hidden state
        h_t = o * tf.keras.backend.tanh(C_t)

        return h_t, [h_t, C_t]

In [ ]:
input_dim = 10  # number of input features
timesteps = 5  # number of time steps
units = 20  # number of LSTM units
dropout_rate = 0.5  # dropout rate

inputs = tf.keras.Input(shape=(timesteps, input_dim))
lstm_cell = LSTMCellWithDropout(units, dropout_rate=dropout_rate)
rnn_layer = layers.RNN(lstm_cell, return_sequences=True)
outputs = rnn_layer(inputs)

model = models.Model(inputs, outputs)
model.summary()

In [ ]:
# Create some sample data
X = np.random.random((1, timesteps, input_dim))

# Make predictions
output = model.predict(X)
print(output)